In [1]:
!pip install tensorflow
!pip install numpy

In [2]:
# Célula 1: Configuração e Definição dos Dados

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Frases de exemplo com dependência de longo prazo
# Estas frases são projetadas para que a última palavra dependa de uma palavra anterior na sequência.
text_data = [
    "O gato que estava no telhado, pulou para a árvore e depois de um tempo, miou.",
    "O cachorro que correu pelo parque, brincou com a bola e finalmente, latia.",
    "A criança que jogava no jardim, encontrou um brinquedo e começou a chorar.",
    "O gato que comeu a ração, bebeu água e depois de um cochilo, miou.",
    "O cachorro que farejou o chão, encontrou um osso e em seguida, latia.",
    "A criança que desenhava na parede, foi repreendida e por fim, chorar.",
    "O pássaro que voava alto, pousou no galho e então, cantou.",
    "O leão que rugia na savana, caçou sua presa e depois, dormiu."
]

print("Dados de texto carregados.")


Dados de texto carregados.


In [3]:
# Célula 2: Pré-processamento dos Dados

# 1. Tokenização e Criação do Vocabulário
# O Tokenizer converte palavras em IDs numéricos e constrói o vocabulário.
# <unk> é um token para palavras "out-of-vocabulary" (fora do vocabulário).
tokenizer = Tokenizer(num_words=None, oov_token="<unk>")
tokenizer.fit_on_texts(text_data) # Constrói o vocabulário a partir dos dados de texto
word_index = tokenizer.word_index # Mapeamento palavra -> ID
vocab_size = len(word_index) + 1 # +1 para o token <unk>

print(f"Tamanho do vocabulário: {vocab_size}")
print(f"Mapeamento de palavras (amostra): {list(word_index.items())[:10]}...") # Mostra as 10 primeiras

# 2. Criar Sequências de Treinamento
# Para cada frase, geramos múltiplas sequências de entrada/saída.
# Ex: "O gato miou." -> ("O", "gato"), ("O gato", "miou")
input_sequences = []
for line in text_data:
    # Converte a frase em uma sequência de IDs numéricos
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        # Cria n-gramas (sequências crescentes)
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# 3. Padding das Sequências
# Garante que todas as sequências de entrada tenham o mesmo comprimento,
# o que é necessário para a entrada da rede neural.
max_sequence_len = max([len(x) for x in input_sequences])
padded_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# 4. Separar Entrada (X) e Saída (y)
# X: todas as palavras da sequência, exceto a última (que é a entrada para a LSTM)
# y: a última palavra da sequência (que é a saída esperada)
X = padded_sequences[:, :-1]
y = padded_sequences[:, -1]

# 5. One-Hot Encode da Saída
# Converte os IDs numéricos da saída em vetores one-hot,
# onde cada posição representa uma palavra no vocabulário.
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

print(f"\nComprimento máximo da sequência: {max_sequence_len}")
print(f"Formato de X (entrada): {X.shape}")
print(f"Formato de y (saída one-hot): {y.shape}")
print(f"\nExemplo de X (entrada para a LSTM):\n{X[0]}")
print(f"Exemplo de y (saída one-hot esperada):\n{y[0]}")


Tamanho do vocabulário: 65
Mapeamento de palavras (amostra): [('<unk>', 1), ('que', 2), ('e', 3), ('o', 4), ('a', 5), ('um', 6), ('no', 7), ('depois', 8), ('gato', 9), ('de', 10)]...

Comprimento máximo da sequência: 16
Formato de X (entrada): (96, 15)
Formato de y (saída one-hot): (96, 65)

Exemplo de X (entrada para a LSTM):
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 4]
Exemplo de y (saída one-hot esperada):
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [4]:
# Célula 3: Construção e Compilação do Modelo LSTM

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Dimensões do Embedding
# O embedding_dim define o tamanho do vetor denso que representará cada palavra.
embedding_dim = 100

# Construção do Modelo Sequencial
model = Sequential([
    # Camada de Embedding: Converte IDs de palavras em vetores densos.
    # vocab_size: número total de palavras únicas.
    # embedding_dim: dimensão dos vetores de palavras.
    # input_length: comprimento das sequências de entrada (max_sequence_len - 1, pois a última é a saída).
    Embedding(vocab_size, embedding_dim, input_length=max_sequence_len-1),

    # Camada LSTM: A camada principal que processa a sequência e mantém a "memória".
    # 150: número de unidades/neurônios na camada LSTM.
    # return_sequences=False: A camada LSTM retorna apenas a saída do último passo da sequência,
    #                         adequado para tarefas de classificação de sequência para um único rótulo.
    LSTM(150, return_sequences=False),

    # Camada Densa de Saída: Classifica a próxima palavra.
    # vocab_size: número de classes (uma para cada palavra no vocabulário).
    # activation='softmax': Garante que a saída seja uma distribuição de probabilidade sobre o vocabulário.
    Dense(vocab_size, activation='softmax')
])

# Compilação do Modelo
# loss='categorical_crossentropy': Função de perda para classificação multiclasse one-hot encoded.
# optimizer='adam': Otimizador Adam, uma boa escolha padrão para a maioria dos modelos de DL.
# metrics=['accuracy']: Métrica para monitorar o desempenho durante o treinamento.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Exibe um resumo da arquitetura do modelo
model.summary()


/home/glkaiky/Desktop/PucMinas/Ia/Exemplo/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-06-04 08:07:53.735984: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Célula 4: Treinamento do Modelo

print("Iniciando o treinamento do modelo...")

# model.fit: Treina o modelo com os dados de entrada (X) e saída (y).
# epochs: Número de vezes que o modelo verá todo o dataset. 100 épocas são suficientes para este dataset pequeno.
# verbose=1: Exibe o progresso do treinamento em cada época.
history = model.fit(X, y, epochs=100, verbose=1)

print("\nTreinamento concluído.")


Iniciando o treinamento do modelo...
Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.0599 - loss: 4.1718
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1354 - loss: 4.1479
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0977 - loss: 4.1085
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0964 - loss: 4.0189
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0964 - loss: 3.9051
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1250 - loss: 3.8016
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1341 - loss: 3.8813
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1380 - loss: 3.8639
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1562 - loss: 3.7332
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1484 - loss: 3.7141
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1576 - loss: 3.6694
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━

In [6]:
# Célula 5: Função para Prever a Próxima Palavra

def predict_next_word(model, tokenizer, text_sequence, max_len):
    """
    Prevê a próxima palavra em uma sequência de texto.

    Args:
        model: O modelo LSTM treinado.
        tokenizer: O Tokenizer usado para pré-processar os dados de treinamento.
        text_sequence: A sequência de texto de entrada (string).
        max_len: O comprimento máximo das sequências de entrada (max_sequence_len do pré-processamento).

    Returns:
        A palavra prevista como string.
    """
    # Converte a sequência de texto em IDs numéricos
    token_list = tokenizer.texts_to_sequences([text_sequence])[0]
    
    # Faz o padding da sequência para corresponder ao comprimento de entrada do modelo
    # max_len-1 porque a camada de Embedding espera o comprimento da sequência de entrada (X),
    # que é o comprimento máximo - 1 (a última palavra é a saída).
    padded_token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
    
    # Obtém as probabilidades de cada palavra no vocabulário ser a próxima
    predicted_probabilities = model.predict(padded_token_list, verbose=0)[0]
    
    # Encontra o índice da palavra com a maior probabilidade
    predicted_word_index = np.argmax(predicted_probabilities)
    
    # Mapeia o índice de volta para a palavra correspondente
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    
    # Retorna <unk> se a palavra prevista não for encontrada (improvável com nosso vocabulário)
    return "<unk>"

print("Função 'predict_next_word' definida.")


Função 'predict_next_word' definida.


In [7]:
# Célula 6: Teste e Demonstração das Previsões

# Função para prever a frase completa
def predict_full_phrase(model, tokenizer, start_text_sequence, max_input_len, max_output_words=10):
    """
    Prevê a frase completa a partir de uma sequência inicial, palavra por palavra.

    Args:
        model: O modelo LSTM treinado.
        tokenizer: O Tokenizer usado para pré-processar os dados de treinamento.
        start_text_sequence: A sequência de texto inicial (string).
        max_input_len: O comprimento máximo das sequências de entrada para o modelo (max_sequence_len do pré-processamento).
        max_output_words: O número máximo de palavras a serem geradas após a sequência inicial.

    Returns:
        A frase completa gerada.
    """
    generated_phrase = start_text_sequence
    for _ in range(max_output_words):
        token_list = tokenizer.texts_to_sequences([generated_phrase])[0]
        # Pad a sequência para o comprimento de entrada do modelo
        # max_len-1 porque a camada de Embedding espera o comprimento da sequência de entrada (X),
        # que é o comprimento máximo - 1 (a última palavra é a saída).
        padded_token_list = pad_sequences([token_list], maxlen=max_input_len - 1, padding='pre')

        # Preveja a próxima palavra
        predicted_probabilities = model.predict(padded_token_list, verbose=0)[0]
        predicted_word_index = np.argmax(predicted_probabilities)

        predicted_word = ""
        # Mapear índice de volta para palavra
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break
        
        # Se a palavra prevista for um token de fim de frase ou um token desconhecido, pare.
        # Ajuste esta condição conforme os tokens que você espera para o fim da frase.
        if predicted_word == "." or predicted_word == "<unk>" or predicted_word == "":
            break
        
        generated_phrase += " " + predicted_word
    return generated_phrase

# Frases para testar o modelo (apenas o início)
test_phrases = [
    "O gato que estava no telhado",
    "O cachorro que correu pelo parque, brincou com a bola e finalmente",
    "A criança que jogava no jardim, encontrou um brinquedo e começou a",
    "O gato que comeu a ração, bebeu água e depois de um cochilo",
    "O cachorro que farejou o chão, encontrou um osso e em seguida",
    "A criança que desenhava na parede, foi repreendida e por fim",
    "O pássaro que voava alto, pousou no galho e então",
    "O leão que rugia na savana, caçou sua presa e depois"
]

print("\n--- Previsões de Frases Completas ---")
for phrase in test_phrases:
    full_predicted_phrase = predict_full_phrase(model, tokenizer, phrase, max_sequence_len)
    print(f"Início: '{phrase}' -> Frase completa prevista: '{full_predicted_phrase}'")

# Exemplo de uma frase com uma dependência longa e uma palavra-chave no meio
# Esta frase é mais longa e com "distratores" para mostrar a capacidade da LSTM
# de manter a memória do sujeito ("gato") ao longo da sequência.
new_long_phrase = "O gato, que estava dormindo profundamente em sua cama macia, e que sonhava com peixes, agora acordou e vai"
full_predicted_phrase_long = predict_full_phrase(model, tokenizer, new_long_phrase, max_sequence_len)
print(f"\nInício Longo (dependência distante): '{new_long_phrase}' -> Frase completa prevista: '{full_predicted_phrase_long}'")

# Exemplo de outra frase longa
new_long_phrase_2 = "O cachorro, que fugiu da coleira, e correu por toda a rua, agora parou e vai"
full_predicted_phrase_long_2 = predict_full_phrase(model, tokenizer, new_long_phrase_2, max_sequence_len)
print(f"Início Longo (dependência distante): '{new_long_phrase_2}' -> Frase completa prevista: '{full_predicted_phrase_long_2}'")

print("\n--- Fim da Demonstração ---")



--- Previsões de Frases Completas ---
Início: 'O gato que estava no telhado' -> Frase completa prevista: 'O gato que estava no telhado pulou para a árvore e depois de um tempo miou'
Início: 'O cachorro que correu pelo parque, brincou com a bola e finalmente' -> Frase completa prevista: 'O cachorro que correu pelo parque, brincou com a bola e finalmente latia miou miou miou miou miou miou miou miou miou'
Início: 'A criança que jogava no jardim, encontrou um brinquedo e começou a' -> Frase completa prevista: 'A criança que jogava no jardim, encontrou um brinquedo e começou a chorar chorar miou miou miou miou miou miou miou miou'
Início: 'O gato que comeu a ração, bebeu água e depois de um cochilo' -> Frase completa prevista: 'O gato que comeu a ração, bebeu água e depois de um cochilo miou miou miou miou miou miou miou miou miou miou'
Início: 'O cachorro que farejou o chão, encontrou um osso e em seguida' -> Frase completa prevista: 'O cachorro que farejou o chão, encontrou um osso e em